In [162]:
import pandas as pd
import numpy as np
import importlib
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import seaborn as sb
from matplotlib.pyplot import xlabel, ylabel, colorbar
from pypfopt.objective_functions import portfolio_variance

import utilities.utility as utility
import plotly.graph_objects as go
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt import plotting

importlib.reload(utility)

<module 'utilities.utility' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/utility.py'>

In [163]:
df = pd.read_csv('../../data/df_monthly_returns_complete_percentage.csv')
df_overview = pd.read_csv('../../data/df_overview.csv', index_col=0)

In [164]:
'''
df_overview['return_rate_1y_avg'] = df_overview['return_rate_1y_avg'] - 1
df_overview['return_rate_5y_avg'] = df_overview['return_rate_5y_avg'] - 1
df_overview['return_rate_10y_avg'] = df_overview['return_rate_10y_avg'] - 1
df_overview['return_rate_25y_avg'] = df_overview['return_rate_25y_avg'] - 1
'''

In [165]:
df_overview = df_overview.dropna()

In [166]:
df

,Date,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,1999-11-01,1.13,1.00,1.00,1.00,1.03,1.10,1.01,1.03,1.00,...,1.00,1.00,1.00,0.93,1.00,0.99,1.00,1.00,1.00,0.84
1,1999-12-01,1.11,1.00,1.00,1.00,0.92,1.02,1.10,1.00,1.00,...,1.00,1.00,1.00,1.18,1.00,1.01,1.00,1.00,1.00,0.93
2,2000-01-01,1.02,1.00,1.00,1.00,0.96,0.88,0.89,0.73,1.00,...,1.00,1.00,1.00,0.85,1.00,1.22,1.00,1.00,1.00,0.93
3,2000-02-01,1.00,1.01,0.81,1.23,1.08,0.96,1.25,1.95,1.18,...,1.03,0.94,1.07,1.04,1.47,1.97,1.13,0.95,1.01,0.94
4,2000-03-01,1.00,1.00,1.12,1.03,0.98,1.41,0.94,1.00,1.03,...,1.00,1.15,1.07,0.94,1.00,1.03,1.00,1.01,1.46,1.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2024-05-01,0.95,1.10,1.06,1.02,1.14,1.12,1.12,1.06,1.06,...,1.09,1.07,1.16,1.05,0.99,1.08,1.03,0.98,0.86,0.98
295,2024-06-01,1.00,0.96,0.96,0.97,0.95,0.96,0.94,0.94,0.95,...,1.07,0.91,1.18,0.89,1.01,0.90,1.19,1.01,1.14,0.98
296,2024-07-01,1.19,1.08,1.02,1.06,1.12,1.22,1.05,1.09,1.07,...,1.10,1.07,1.06,1.16,0.94,0.96,1.14,1.28,0.90,1.09
297,2024-08-01,0.96,0.78,1.06,1.08,0.98,0.98,1.03,1.07,1.10,...,0.95,1.04,1.16,0.87,0.96,1.00,0.97,0.93,0.89,1.04


## 1. Industry historical return rate

In [167]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Melt the returns dataframe to long format
df_long_returns = df.melt(id_vars=['Date'], var_name='stock_ticker_symbol', value_name='return_rate')

# Merge the returns and industries dataframes
df_merged = pd.merge(df_long_returns, df_overview, on='stock_ticker_symbol')

# Resample to monthly and calculate mean return for each ticker
df_merged.set_index('Date', inplace=True)

df_i_share_ratio = df_merged[['industry', 'return_rate']].fillna(1).groupby(['industry', df_merged.index]).agg({'return_rate': 'mean'})

df_i_share_ratio = df_i_share_ratio.reset_index(level=0)
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)

# Switch returns from multipliable to decimal by removing 1
df_i_share_ratio[['return_rate']] = df_i_share_ratio[['return_rate']] - 1

# Create line plot
fig = px.line(df_i_share_ratio,
              x='Date',
              y='return_rate',
              color='industry',
              title='Industry historical returns',
              labels={'Date': 'Date', 'return_rate': 'Return Rate'})

fig.update_layout(template="plotly_white",
                  yaxis=dict(
                      tickformat='.0%',
                      range=[-0.4, 0.4]
                  ))

# Show the plot
fig.show()

### 1.1 Industry accumulative returns

In [168]:
df_i_share_ratio

,Date,industry,return_rate
0,1999-11-01,Auto Components,0.005556
1,1999-12-01,Auto Components,0.016667
2,2000-01-01,Auto Components,-0.034444
3,2000-02-01,Auto Components,1.494444
4,2000-03-01,Auto Components,0.047778
...,...,...,...
9264,2024-05-01,Utilities,0.118000
9265,2024-06-01,Utilities,-0.018000
9266,2024-07-01,Utilities,0.098000
9267,2024-08-01,Utilities,0.024000


#### Last 25 years

In [169]:
# df_i_share_ratio
min_date = pd.to_datetime(df_i_share_ratio['Date'].values[-1]).replace(day=1) - pd.DateOffset(months=12*25)
min_datestr = min_date.strftime('%Y-%m-%d')
df_i_share_ratio = df_i_share_ratio.loc[df_i_share_ratio['Date'] >= min_datestr]
#
df_cumulative_returns = df_i_share_ratio.copy()
df_cumulative_returns['return_rate'] = df_cumulative_returns['return_rate'] + 1
# move industry in its own column
df_cumulative_returns = df_cumulative_returns.pivot_table(index=['Date'], columns='industry', values=['return_rate'])
# clean columns, "date" will be the index
df_cumulative_returns.columns = [col[1] for col in df_cumulative_returns.columns.values]
# calculate accumulative returns
df_cumulative_returns = df_cumulative_returns.cumprod() - 1
df_cumulative_returns = df_cumulative_returns.reset_index()
# move back to the old structure now containing accumulative return rates instead of return rate
df_cumulative_returns = df_cumulative_returns.melt(id_vars=['Date'], var_name='industry', value_name='cum_return_rate')
# set percentage column
df_cumulative_returns['cum_return_rate_pct'] = df_cumulative_returns['cum_return_rate'] * 100
# plot
fig = px.line(df_cumulative_returns, x='Date',
              y='cum_return_rate_pct', color='industry',
              title='Performance - monthly cumulative returns (last 25 years)',
              labels={'cum_return_rate_pct':'Monthly cumulative returns (%)', })

fig.update_layout(template="plotly_white")
fig.show()

#### Last 5 years

In [170]:
# df_i_share_ratio
min_date = pd.to_datetime(df_i_share_ratio['Date'].values[-1]).replace(day=1) - pd.DateOffset(months=12*5)
min_datestr = min_date.strftime('%Y-%m-%d')
df_i_share_ratio = df_i_share_ratio.loc[df_i_share_ratio['Date'] >= min_datestr]
#
df_cumulative_returns = df_i_share_ratio.copy()
df_cumulative_returns['return_rate'] = df_cumulative_returns['return_rate'] + 1
# move industry in its own column
df_cumulative_returns = df_cumulative_returns.pivot_table(index=['Date'], columns='industry', values=['return_rate'])
# clean columns, "date" will be the index
df_cumulative_returns.columns = [col[1] for col in df_cumulative_returns.columns.values]
# calculate accumulative returns
df_cumulative_returns = df_cumulative_returns.cumprod() - 1
df_cumulative_returns = df_cumulative_returns.reset_index()
# move back to the old structure now containing accumulative return rates instead of return rate
df_cumulative_returns = df_cumulative_returns.melt(id_vars=['Date'], var_name='industry', value_name='cum_return_rate')
# set percentage column
df_cumulative_returns['cum_return_rate_pct'] = df_cumulative_returns['cum_return_rate'] * 100
# plot
fig = px.line(df_cumulative_returns, x='Date',
              y='cum_return_rate_pct', color='industry',
              title='Performance - monthly cumulative returns (last 5 years)',
              labels={'cum_return_rate_pct':'Monthly cumulative returns (%)', })
fig.update_layout(template="plotly_white")
fig.show()

## 2. Industry correlation

In [171]:
df_i_share_ratio

,Date,industry,return_rate
238,2019-09-01,Auto Components,0.054444
239,2019-10-01,Auto Components,0.065556
240,2019-11-01,Auto Components,0.107778
241,2019-12-01,Auto Components,0.025556
242,2020-01-01,Auto Components,-0.104444
...,...,...,...
9264,2024-05-01,Utilities,0.118000
9265,2024-06-01,Utilities,-0.018000
9266,2024-07-01,Utilities,0.098000
9267,2024-08-01,Utilities,0.024000


In [172]:
industry_size = df_overview.groupby(['industry']).size().reset_index(name='Count')
industry_size

,industry,Count
0,Auto Components,9
1,Automobiles,1
2,Banks,5
3,Building Products,5
4,Chemicals,2
5,Commercial Services,17
6,Consumer Durables,12
7,Consumer Services,9
8,Containers & Packaging,3
9,Diversified Financials,13


### Industry size

In [173]:
fig = px.bar(industry_size, x='industry', y='Count', color_discrete_sequence=[px.colors.qualitative.Plotly[2]])

fig.update_layout(
    title='Number of companies per industry',
    xaxis=dict(
        title='Industries'
    ),
    yaxis=dict(
        title='Number of companies'
    ),
    showlegend=False,
    template='plotly_white',
)

### 2.1 Correlation matrix

In [174]:
# Pivot the DataFrame so that each industry is a column and the rows represent dates
df_industry_heatmap = df_i_share_ratio[['Date', 'industry', 'return_rate']].pivot(index='Date', columns='industry', values='return_rate')

# Ensure the columns are in a consistent order (optional)
df_industry_heatmap = df_industry_heatmap.sort_index(axis=1)

# Calculate the correlation matrix
correlation_matrix = df_industry_heatmap.corr()

In [175]:
# Convert correlation matrix to 1-D Series and sort 
sorted_mat = correlation_matrix.unstack().sort_values()
sorted_mat

industry                       industry                     
Transportation Infrastructure  Automobiles                     -0.197892
Automobiles                    Transportation Infrastructure   -0.197892
Telecommunication Services     Automobiles                      0.025751
Automobiles                    Telecommunication Services       0.025751
Refiners & Pipelines           Automobiles                      0.071086
                                                                  ...   
Telecommunication Services     Telecommunication Services       1.000000
Textiles & Apparel             Textiles & Apparel               1.000000
Traders & Distributors         Traders & Distributors           1.000000
Homebuilders                   Homebuilders                     1.000000
Utilities                      Utilities                        1.000000
Length: 961, dtype: float64

In [176]:
# Create the heatmap
fig = px.imshow(correlation_matrix,
                # labels=dict(x="Industry", y="Industry", color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                color_continuous_scale="RdBu",
                title="Industry Correlation Heatmap")

fig.layout.height = 1000
fig.layout.width = 1250

# Show the figure
fig.show()

### 2.1 Correlation of "Software & Services" to other industries

In [177]:
fig = px.bar(correlation_matrix.loc[correlation_matrix.index !='Software & Services', ['Software & Services']])

fig.update_layout(
    title='Correlation of "Software & Services" with other industries',
    xaxis=dict(
        title='Industries'
    ),
    yaxis=dict(
        title='Correlation Level'
    ),
    showlegend=False,
    template='plotly_white',
)